encoder末尾全连接层（fc1）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import sys
# 获取当前文件所在的文件夹路径
if "__file__" in globals():
    # 获取__file__变量的值
    file_path = __file__
    # 获取当前文件所在的文件夹路径
    dir_now = os.path.dirname(file_path)
else:
    # 获取当前工作目录
    dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # 防止jupyter爆内存
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_PLUTO
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
quantize_lut = True
# for method in [METHOD_MITHRAL, METHOD_PQ]:

In [3]:
linear_name = 'fc1'
feedback_bits = 256
linear_name_full = "fc1"

auto_train_change_nbits = False # 是否根据已运行的训练性能结果改变nbits自动训练，（train_sam_num取已训练的最大值）
auto_train_change_upcast = False # 是否根据已运行的训练性能结果改变upcast自动训练，（train_sam_num取已训练的最大值）

if auto_train_change_nbits:
    nbits_trained = 8
if auto_train_change_upcast:
    upcast_trained = 16
nbits_goal = 8
upcast_goal = 16
if quantize_lut == False:
    nbits_goal = 0
nbits = nbits_goal # 要运行的量化比特数
upcast_every = upcast_goal # 要运行的upcast

test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)

if not auto_train_change_nbits and not auto_train_change_upcast:
    ncodebooks = 256 # max:512
    ncentroids = 256
    train_sam_num = 650 # 训练集样本数
elif auto_train_change_nbits:
    param2change = "nbits"
    param_trained = nbits_trained
    param_goal = nbits_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "upcast_every", 16)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])
elif auto_train_change_upcast:
    param2change = "upcast_every"
    param_trained = upcast_trained
    param_goal = upcast_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "nbits", nbits_goal)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])

batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
AMM_train_dirs = get_AMM_train_dirs(linear_name, linear_name_full, method, feedback_bits, train_sam_num, test_sam_num)
create_dir(AMM_train_dirs["dir_result"])


In [5]:
dataset_prepare(AMM_train_dirs["dir_joined"], linear_name_full, feedback_bits, [train_sam_num, test_sam_num], 
                batch_size, S1 = S1_dict[linear_name])

查找以此为前缀的数据集： fc1in_train_f256
有比输入样本数更大的数据集，从中提取新数据集
生成的数据集前缀:  fc1in train
原数据集大小:  (224000, 2048)
提取后数据集大小:  (20800, 2048)
查找以此为前缀的数据集： fc1out_train_f256
有比输入样本数更大的数据集，从中提取新数据集
生成的数据集前缀:  fc1out train
原数据集大小:  (224000, 256)
提取后数据集大小:  (20800, 256)


In [6]:
if method == METHOD_PLUTO:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut, 
                        upcast_every=upcast_every, bias_path=AMM_train_dirs["biaspath"])
else:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every)

running method:  Mithral
X.shape:  (20800, 2048)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    0.2991740107536316
learn_multisplits(): returning loss:  0.05627229084521446
learn_multisplits(): initial loss:    3.359151840209961
learn_multisplits(): returning loss:  0.75660998074585
learn_multisplits(): initial loss:    0.4724603295326233


/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../clusterize.py:134: UserWarning: Persisting input arguments took 0.51s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  return optimal_split_val(


learn_multisplits(): returning loss:  0.08874702100411014
learn_multisplits(): initial loss:    0.17834880948066711
learn_multisplits(): returning loss:  0.03785044469930199
learn_multisplits(): initial loss:    0.22982165217399597
learn_multisplits(): returning loss:  0.03341400257207283
learn_multisplits(): initial loss:    0.3045293986797333
learn_multisplits(): returning loss:  0.06144609204915241
learn_multisplits(): initial loss:    0.12116643786430359
learn_multisplits(): returning loss:  0.026105780972230264
learn_multisplits(): initial loss:    0.13698318600654602
learn_multisplits(): returning loss:  0.030469580646922623
learn_multisplits(): initial loss:    0.7667447328567505
learn_multisplits(): returning loss:  0.12267451205627822
learn_multisplits(): initial loss:    6.954737663269043
learn_multisplits(): returning loss:  1.7789880662767246
learn_multisplits(): initial loss:    0.5875802040100098
learn_multisplits(): returning loss:  0.14755313807228276
learn_multisplits(

/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../clusterize.py:134: UserWarning: Persisting input arguments took 0.73s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  return optimal_split_val(


learn_multisplits(): returning loss:  0.14107202028935717
learn_multisplits(): initial loss:    0.2007468044757843
learn_multisplits(): returning loss:  0.060281659782674524
learn_multisplits(): initial loss:    0.23140522837638855
learn_multisplits(): returning loss:  0.054308675436672615
learn_multisplits(): initial loss:    0.573783814907074
learn_multisplits(): returning loss:  0.12497808021328183
learn_multisplits(): initial loss:    0.13783694803714752
learn_multisplits(): returning loss:  0.04570079526169479
learn_multisplits(): initial loss:    0.15732510387897491
learn_multisplits(): returning loss:  0.049729685836041426
learn_multisplits(): initial loss:    1.2720448970794678


...

/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../clusterize.py:134: UserWarning: Persisting input arguments took 0.71s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  return optimal_split_val(


learn_multisplits(): returning loss:  208.04948867764324
learn_multisplits(): initial loss:    149.28152465820312


/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../clusterize.py:134: UserWarning: Persisting input arguments took 0.66s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  return optimal_split_val(


learn_multisplits(): returning loss:  33.77085117386741
learn_multisplits(): initial loss:    32.080142974853516
learn_multisplits(): returning loss:  9.899177164548746
X_res mse / X mse:  0.24933168
fitting dense lstsq to X_res
  with X_enc:(20800, 256) Y:(20800, 2048)
fitted dense lstsq with W:(65536, 2048)
X_res mse / X mse after lstsq:  0.40351063


/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../vquantizers.py:640: UserWarning: Persisting input arguments took 1.92s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  self.splits_lists, self.centroids = clusterize.learn_mithral(


In [7]:
x_test = np.load(AMM_train_dirs["dir_test"]+'/'+AMM_train_dirs["linearin_path_test"])
w_test = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["weightpath"])
bias = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["biaspath"])
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
if method == METHOD_PLUTO:
    y_out_last = y_out_matmul
else:
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), 
                                                        y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL or method == METHOD_PQ or method == METHOD_PLUTO or method == METHOD_MITHRALPQ:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every)), y_out_last_re)
else:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids)), y_out_last_re)
    # np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%iori.npy' % 
    #                                                     (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
    #                                                     ncodebooks, ncentroids)), y_out_last)

[[ 0.05257927 -0.41137594  0.3493459  ... -0.335446    0.11707558
  -0.11652333]
 [-0.07266535 -0.03564208 -0.7778557  ... -0.46069062 -0.32128057
  -0.17914565]
 [-0.01004304  0.02698022  0.16147895 ... -0.2102014   0.17969789
  -0.05390103]
 ...
 [-0.13528766  0.21484716 -0.6526111  ...  0.10291016 -0.2586583
   0.07134359]
 [ 0.36569083  0.15222485 -0.52736646 ...  0.04028785 -0.07079136
   0.13396591]
 [-0.5110215   0.02698022 -0.3394995  ... -0.08495677 -0.2586583
  -0.24176796]]
y_out_last.shape:  (32000, 256)
y_out_last_re.shape:  (1000, 32, 256)
